In [2]:
from bs4 import BeautifulSoup
import urllib
from fake_useragent import UserAgent
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
from textblob import TextBlob
import re

In [17]:
def Anime():
    
    def popular_anime(url):
        
    #Fucntion to scrape popular anime of current time from crunchyroll website and will store it in a dataframe
    
        anime=pd.DataFrame(np.zeros(120).reshape((40,3)),columns=['Anime_Name',"Number of Videos Aired",'Link'])
     
        driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
        driver.get(url)
    
        soup=BeautifulSoup(driver.page_source,'lxml')

        content=soup.find('div',{'id':'main_content'})
    
        i=0
    
        for div in content.find_all('a',class_='portrait-element block-link titlefix'):
        
            name=div.find('span',class_='series-title block ellipsis')
            no_videos=div.find('span',class_='series-data block ellipsis')
        
            anime['Anime_Name'][i]=name.text.strip()
            anime['Number of Videos Aired'][i]=no_videos.text.strip()
            anime['Link'][i]=div['href']
            i=i+1
        
            
        driver.quit()
        
        return anime

    url_popular_anime='https://www.crunchyroll.com/videos/anime'
    anime_list=popular_anime(url_popular_anime)

    #To get the proper link of each webpage
    for i in range(len(anime_list)):
        temp=anime_list['Link'][i]
        anime_list['Link'][i]='https://www.crunchyroll.com'+temp
    

    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    anime_list['Reviews']=anime_list['Anime_Name']

    
    #To scrape the reviews of each anime which will be used to determine the sentiment of them
    for j in range(len(anime_list)):
    
        url=anime_list['Link'][j]
        title=anime_list['Anime_Name']
    
        l=[]
    
        for i in range(1,11):
        
            url_reviews=url+'/reviews/helpful/page{}'.format(i)
            driver.get(url_reviews)
            soup=BeautifulSoup(driver.page_source,'lxml')

            content=soup.find('div',class_='user-reviews')
        
            for x in content.find_all('li',class_='white-wrapper container-shadow medium-margin-bottom'):
                review=x.find('div',class_='strong left')
                l.append(review.text)
                
            
        anime_list['Reviews'][j]=l
    
    anime_list.to_csv('Anime.csv')
    anime_list=pd.read_csv('Anime.csv')
    anime_list.drop('Unnamed: 0',axis=1,inplace=True)

    
    #Cleaning the text from unwanted characters
    for i in range(len(anime_list)):
        temp=anime_list['Reviews'][i]
        anime_list['Reviews'][i]=temp.replace('/','').replace(',','').replace('" \'','').replace('.','').replace('-','').replace('?','').replace('" \'','').replace('\'','').replace('[','').replace(']','').replace('"','')
    
    anime_list['Polarity Score']=anime_list['Anime_Name']
    anime_list['Sentiment']=anime_list['Anime_Name']

    
    #Determining sentiment score using TextBlob and classifying them according to their score
    for i in range(len(anime_list)):
        temp=TextBlob(anime_list['Reviews'][i])
        score=temp.sentiment.polarity
        anime_list['Polarity Score'][i]=score
        if score>0.5:
            anime_list['Sentiment'][i]="Very Positive Reviews"
        elif(score>0.3 and score<0.5):
            anime_list['Sentiment'][i]='Positive Reviews' 
        elif(score>0 and score<0.3):
            anime_list['Sentiment'][i]='Mixed Reviews'
        else:
            anime_list['Sentiment'][i]='Negative Reviews'
            
            
            
    #Scraping the imdb links from google search then scraping the storyline and rating of each anime
    imdb_links=[]
    for i in range(len(anime_list)):

        query = anime_list['Anime_Name'][i]
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 1

        ua = UserAgent()

        google_url = "https://www.google.com/search?q=" + query +"imdb"+"&num=" + str(number_result)
        response = requests.get(google_url, {"User-Agent": ua.random})
        soup = BeautifulSoup(response.text, "html.parser")

        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})


        for r in result_div:
            # Checks if each element is present, else, raise exception
            try:
                link = r.find('a', href = True)
                title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
                #description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

                # Check to make sure everything is present before appending
                if link != '': 
                    imdb_links.append(link['href'][7:44])
                    #titles.append(title)
                    #descriptions.append(description)
            # Next loop if one element is not present
                break
            except:
                continue
                
        anime_list['Storyline']=anime_list['Link']
        anime_list['Rating']=anime_list['Link']
        
        for i in range(len(imdb_links)):
            
            response=requests.get(imdb_links[i])
            soup=BeautifulSoup(response.text,'lxml')
        
            if(re.search('https://www.imdb.com',imdb_links[i])):
                for j in soup.select('#titleStoryLine div:nth-child(3) p span'):
                    anime_list['Storyline'][i]=j.text
                for k in soup.select('#title-overview-widget div.vital div.title_block div div.ratings_wrapper div.imdbRating div strong span'):
                    anime_list['Rating'][i]=k.text
            else:
                anime_list['Storyline'][i]='Not Available'
                anime_list['Rating'][i]='Not Available'
                continue

    anime_list.to_csv('Anime_Final.csv')
    
    return anime_list

In [4]:
def Manga():
    
    def popular_manga(url):
        
        #Function to scrape popular manga of current time
    
        driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
        manga = pd.DataFrame(np.zeros(98).reshape((49,2)),columns=['Manga Name','Link'])
    
        driver.get(url)
    
        soup=BeautifulSoup(driver.page_source,'lxml')
    
        content2=soup.find('div',{'id':'main_content'})
    
        i=0
    
        for x in content2.find_all('a',class_='portrait-element block-link titlefix'):
            title=x.find('span',class_='series-title block ellipsis')
            manga['Manga Name'][i]=title.text
            manga['Link'][i]='https://www.crunchyroll.com'+x['href']
            i=i+1
            #print(title.text)
    
        return manga
    
    url_popular_manga='https://www.crunchyroll.com/comics/manga'
    manga_list=popular_manga(url_popular_manga)
    
      
    
    #To scrape the overview of each scraped manga
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    manga_list['Overview']=manga_list['Manga Name']
    for i in range(len(manga_list)):
        temp_url=manga_list['Link'][i]
        driver.get(temp_url)
        soup=BeautifulSoup(driver.page_source,'lxml')
        content9=soup.find('div',class_='right')
    
        if(content9==None):
            manga_list['Overview'][i]='Not Available'
        
        else:
            for y in content9.find_all('p',class_='description'):
            
                manga_list['Overview'][i]=y.text.strip()
                
    return manga_list
                

In [15]:
def recommendation_Manga(manga_list):
    
    #Used to convert text into computer readable form
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    manga_list['Overview'] = manga_list['Overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(manga_list['Overview'])
    
    manga_list['ID']=manga_list['Manga Name']
    manga_list['ID']=manga_list.index+1000
    
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_distance = cosine_similarity(tfidf_matrix,tfidf_matrix)
    
    return similarity_distance

def recommendation_Anime(anime_list):
    
    #Used to convert text into computer readable form
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    
    tfidf_matrix = tfidf.fit_transform(anime_list['Storyline'])
    
    anime_list['ID']=anime_list.index+1000
    
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_distance = cosine_similarity(tfidf_matrix,tfidf_matrix)
    
    return similarity_distance

def get_recommendation_manga(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx  = manga['ID'][manga['Manga Name']==title].index[0]

    # Get all manga with same similarity score
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the manga based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the manga indices
    manga_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar manga
    return manga.iloc[manga_indices]



def get_recommendation_anime_all(anime_list,sim_distance):
    
    def get_recommendation_anime(title, cosine_sim):
        # Get the index of the movie that matches the title
        idx  = anime['ID'][anime['Anime_Name']==title].index[0]

        # Get all manga with same similarity score
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort the manga based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the manga indices
        anime_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar manga
        return anime.iloc[anime_indices]
    
    anime_list['Recommended']=anime_list['ID']
        
    for i in range(len(anime_list)):
        q=[]
        df2=get_recommendation_anime(anime_list['Anime_Name'][i],sim_distance)
        temp=df2['Anime_Name'][:5]
        q.append(temp.values)
        anime_list['Recommended'][i]=q
    return anime_list

In [6]:
def latest_news(url):
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
    driver.get(url)
    
    soup=BeautifulSoup(driver.page_source,'lxml')
    
    content=soup.find('ul',class_='news')
    
    for b in content.find_all('li',class_='news-item'):
        headline=b.find('h2')
        print(headline.text)

In [8]:
anime=Anime()
anime

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Anime_Name,Number of Videos Aired,Link,Reviews,Polarity Score,Sentiment,Storyline,Rating
0,RADIANT,38 Videos,https://www.crunchyroll.com/radiant,Read the Manga if you want the FULL experience...,0.195576,Mixed Reviews,Synopsis Seth is an aspiring sorcerer from...,6.6
1,BORUTO: NARUTO NEXT GENERATIONS,141 Videos,https://www.crunchyroll.com/boruto-naruto-next...,If Boruto follows Naruto in terms of episodes ...,-0.00561858,Negative Reviews,Naruto was a young shinobi with an incorri...,7.2
2,Naruto Shippuden,500 Videos,https://www.crunchyroll.com/naruto-shippuden,Sorry Naruto you deserved better An amazing ri...,0.237361,Mixed Reviews,"Naruto Uzumaki, is a loud, hyperactive, ad...",8.5
3,JoJo's Bizarre Adventure,77 Videos,https://www.crunchyroll.com/jojos-bizarre-adve...,SONO CHI NO SADAME!! A faithful adaptation of ...,0.41767,Positive Reviews,Chronicles the struggles of the cursed Joe...,8.4
4,DARLING in the FRANXX,120 Videos,https://www.crunchyroll.com/darling-in-the-franxx,A Unique and Telling Twist on Our Own Modern R...,0.340755,Positive Reviews,The story is set in the distant future. Th...,7.3
5,The Ancient Magus' Bride,108 Videos,https://www.crunchyroll.com/the-ancient-magus-...,i REALLY like this but a lot of you have the w...,0.474579,Positive Reviews,"Chise Hatori is 15. She, for some reason, ...",7.9
6,Monster Strike,160 Videos,https://www.crunchyroll.com/monster-strike,Unfulfilled plot premise Helps me fall asleep ...,-0.121522,Negative Reviews,"Ren Homura, a middle school student who mo...",5.0
7,ReLIFE,35 Videos,https://www.crunchyroll.com/relife,Ever feel like repeating a year in high school...,0.421057,Positive Reviews,"The story follows Kaizaki Arata, a 27-year...",7.7
8,Eyeshield 21,145 Videos,https://www.crunchyroll.com/eyeshield-21,Definetly my favorite anime Eyeshieldo kun ich...,0.519176,Very Positive Reviews,Kobayakawa Sena is a little guy who always...,7.7
9,How to Keep a Mummy,12 Videos,https://www.crunchyroll.com/how-to-keep-a-mummy,Almost ignored due to premiseglad I didntFor w...,0.443642,Positive Reviews,One day high school student Sora Kashiwagi...,7.6


In [8]:
manga=Manga()
manga

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Manga Name,Link,Overview
0,Domestic Girlfriend,https://www.crunchyroll.com/comics/manga/domes...,High schooler Natsuo is hopelessly in love wit...
1,The Seven Deadly Sins,https://www.crunchyroll.com/comics/manga/the-s...,When they were accused of trying to overthrow ...
2,Attack on Titan,https://www.crunchyroll.com/comics/manga/attac...,"A century ago, the grotesque giants known as T..."
3,EDENS ZERO,https://www.crunchyroll.com/comics/manga/edens...,Eleven months after the grand finale of Fairy ...
4,Knight's & Magic,https://www.crunchyroll.com/comics/manga/knigh...,A genius programmer and hardcore robot otaku i...
5,Inside Mari,https://www.crunchyroll.com/comics/manga/insid...,"A young man is a shut-in, with nothing to do b..."
6,One Room of Happiness,https://www.crunchyroll.com/comics/manga/one-r...,"That day, she was abducted. But to her, that a..."
7,Fairy Tail,https://www.crunchyroll.com/comics/manga/fairy...,Cute girl wizard Lucy wants to join the Fairy ...
8,Tales of Wedding Rings,https://www.crunchyroll.com/comics/manga/tales...,Kekkon Yubiwa Monogatari - A high school girl ...
9,"Lofty Flower, fall for me!!",https://www.crunchyroll.com/comics/manga/lofty...,Shiraishi Kouta has the brains and the looks. ...


In [10]:
similarity_distance=recommendation_Manga(manga)
results=get_recommendation_manga('Inside Mari',similarity_distance)
results=results.reset_index()
results.drop('index',axis=1,inplace=True)
results=results.iloc[1:10,]
results

,Manga Name,Link,Overview,ID
1,Silver Nina,https://www.crunchyroll.com/comics/manga/silve...,Shimazaki Shutaro goes back home to the countr...,1031
2,Hoshi no Samidare: The Lucifer and Biscuit Hammer,https://www.crunchyroll.com/comics/manga/hoshi...,Asamiya Yuuhi was an ordinary college student....,1046
3,One Room of Happiness,https://www.crunchyroll.com/comics/manga/one-r...,"That day, she was abducted. But to her, that a...",1006
4,Girl May Kill,https://www.crunchyroll.com/comics/manga/girl-...,A port town where the Mafia actively operates ...,1041
5,Domestic Girlfriend,https://www.crunchyroll.com/comics/manga/domes...,High schooler Natsuo is hopelessly in love wit...,1000
6,The Daily Life of Crunchyroll-Hime,https://www.crunchyroll.com/comics/manga/the-d...,A bi-monthly Japanese style yonkoma comedy man...,1017
7,To Your Eternity,https://www.crunchyroll.com/comics/manga/to-yo...,A new manga from the creator of the acclaimed ...,1038
8,Crossing Time,https://www.crunchyroll.com/comics/manga/cross...,This is a collection of slice-of-life omnibus ...,1021
9,Talentless Nana,https://www.crunchyroll.com/comics/manga/talen...,An academy on an island in unnavigable waters....,1013


In [16]:
similarity_distance=recommendation_Anime(anime)
results=get_recommendation_anime_all(anime,similarity_distance)
results

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Anime_Name,Number of Videos Aired,Link,Reviews,Polarity Score,Sentiment,Storyline,Rating,ID,Recommended
0,RADIANT,38 Videos,https://www.crunchyroll.com/radiant,Read the Manga if you want the FULL experience...,0.195576,Mixed Reviews,Synopsis Seth is an aspiring sorcerer from...,6.6,1000,"[[RADIANT, Miss Monochrome - The Animation, Ho..."
1,BORUTO: NARUTO NEXT GENERATIONS,141 Videos,https://www.crunchyroll.com/boruto-naruto-next...,If Boruto follows Naruto in terms of episodes ...,-0.00561858,Negative Reviews,Naruto was a young shinobi with an incorri...,7.2,1001,"[[BORUTO: NARUTO NEXT GENERATIONS, Naruto Ship..."
2,Naruto Shippuden,500 Videos,https://www.crunchyroll.com/naruto-shippuden,Sorry Naruto you deserved better An amazing ri...,0.237361,Mixed Reviews,"Naruto Uzumaki, is a loud, hyperactive, ad...",8.5,1002,"[[Naruto Shippuden, BORUTO: NARUTO NEXT GENERA..."
3,JoJo's Bizarre Adventure,77 Videos,https://www.crunchyroll.com/jojos-bizarre-adve...,SONO CHI NO SADAME!! A faithful adaptation of ...,0.41767,Positive Reviews,Chronicles the struggles of the cursed Joe...,8.4,1003,"[[JoJo's Bizarre Adventure, 11eyes, SKET Dance..."
4,DARLING in the FRANXX,120 Videos,https://www.crunchyroll.com/darling-in-the-franxx,A Unique and Telling Twist on Our Own Modern R...,0.340755,Positive Reviews,The story is set in the distant future. Th...,7.3,1004,"[[DARLING in the FRANXX, Bloodivores, Mushibug..."
5,The Ancient Magus' Bride,108 Videos,https://www.crunchyroll.com/the-ancient-magus-...,i REALLY like this but a lot of you have the w...,0.474579,Positive Reviews,"Chise Hatori is 15. She, for some reason, ...",7.9,1005,"[[The Ancient Magus' Bride, Ixion Saga DT, Rec..."
6,Monster Strike,160 Videos,https://www.crunchyroll.com/monster-strike,Unfulfilled plot premise Helps me fall asleep ...,-0.121522,Negative Reviews,"Ren Homura, a middle school student who mo...",5.0,1006,"[[Monster Strike, Future Card Buddyfight, Reco..."
7,ReLIFE,35 Videos,https://www.crunchyroll.com/relife,Ever feel like repeating a year in high school...,0.421057,Positive Reviews,"The story follows Kaizaki Arata, a 27-year...",7.7,1007,"[[ReLIFE, Recovery of an MMO Junkie, Myself; Y..."
8,Eyeshield 21,145 Videos,https://www.crunchyroll.com/eyeshield-21,Definetly my favorite anime Eyeshieldo kun ich...,0.519176,Very Positive Reviews,Kobayakawa Sena is a little guy who always...,7.7,1008,"[[Eyeshield 21, Nyarko-san: Another Crawling C..."
9,How to Keep a Mummy,12 Videos,https://www.crunchyroll.com/how-to-keep-a-mummy,Almost ignored due to premiseglad I didntFor w...,0.443642,Positive Reviews,One day high school student Sora Kashiwagi...,7.6,1009,"[[How to Keep a Mummy, Onee-chan ga Kita, Shug..."


In [13]:
url_news='https://www.crunchyroll.com/news'
latest_news(url_news)


BanG Dream! Three New Singles Have Shipped 150,000 Units in Total


"Precious Pair Edition" Watch Featuring Sailor Uranus & Neptune Available for Pre-orders


Azumi Waki Appears in "Citrus" Costume at Her Solo Debut Single Release Event 


New Anime Film Adaptation of Tatsuya Miyanishi's "Tyrannosaurus" Picture Book Slated for Early Summer 2020


Sword Boys Show Off Their Cutest Forms in Sanrio x Touken Ranbu Collaboration


Keep Your Hands off Eizouken! Live-Action Film's Shooting Wrapped Up


Code Vein Heats Up with Hellfire Knight Content in DLC Launch Trailer 


Hana Yori Dango Brings Rich Anime Romance to Crunchyroll's Catalog


Eiichiro Oda to Oversee Netflix's Live-Action One Piece Series


Nintendo Lawsuit Takes Mario Kart off Japan's Roads


Keep Your Hands off Eizouken! Manga Sales Skyrocket Thanks to New Anime


Shin Sakura Wars Anime Showcased in First Promo, Key Visual


Life After Re:Mind: A Personal Post-Mortem on Kingdom Hearts III


The Forgotten Dragon Ball Game That

In [15]:
anime.shape

(40, 9)

In [14]:
anime.to_csv('Anime_List.csv')
manga.to_csv('Manga_List.csv')